In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H
def gnrt_Hd():
  qnum = 3
  Hd = np.zeros((2**qnum, 2**qnum)).astype(np.complex128)
  Hd += add_in_target(qnum, 0, GateX)@add_in_target(qnum, 1, GateX) + add_in_target(qnum, 0, GateY)@add_in_target(qnum, 1, GateY)
  # Hd += add_in_target(qnum, 1, GateX)@add_in_target(qnum, 2, GateX) + add_in_target(qnum, 1, GateY)@add_in_target(qnum, 2, GateY)
  # Hd += add_in_target(qnum, 2, GateX)@add_in_target(qnum, 0, GateX) + add_in_target(qnum, 2, GateY)@add_in_target(qnum, 0, GateY)
  return Hd

In [3]:
gnrt_Hd().real.astype(int)

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr


In [5]:
for i in range(8):
  B = np.zeros(8)
  B[i] = 1
  C = gnrt_Hd()
  print(first_nonzero_index(B), 5)
  print(first_nonzero_index(C.dot(B)), 5)
  print("====")

000 5
None 5
====
001 5
None 5
====
010 5
100 5
====
011 5
101 5
====
100 5
010 5
====
101 5
011 5
====
110 5
None 5
====
111 5
None 5
====


In [6]:
import numpy as np
from functools import reduce

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

# 定义计算哈密顿量的函数
def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term3 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) for j in range(n)])
        term4 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

# 例子
v = np.array([[0, 1, 1, 0, 0], 
              [0, 1, 0, 1, 0], 
              [0, 0, 0, 0, 1]])
w = np.array([[0, 0, 0, 0, 0], 
              [1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0]])

hamiltonian = calculate_hamiltonian(v, w)
print("Hamiltonian:")
np.set_printoptions(edgeitems=np.inf, linewidth=1000)
print(hamiltonian.real.astype(int))


Hamiltonian:
[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0

In [17]:
import sys
sys.path.append('../')
import zlibrary.linear_system as ls
C = np.array([[-1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
              [-1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
              [0, -1, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, -1, 0, 0, 0, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])
u = ls.find_basic_solution(C)
v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
num = len(v[0])
print("ccc =c")
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  ans = first_nonzero_index(C.dot(B), num)
  if ans != None:
    print(first_nonzero_index(B, num))
    print(ans)
    print("====")

ccc =c
0000000000
1000001010
====
0000000001
1000001011
====
0000000100
1000001110
====
0000000101
1000001111
====
0000010000
1000011010
====
0000010001
1000011011
====
0000010100
1000011110
====
0000010101
1000011111
====
0000100000
1000101010
====
0000100001
1000101011
====
0000100100
1000101110
====
0000100101
1000101111
====
0000110000
1000111010
====
0000110001
1000111011
====
0000110100
1000111110
====
0000110101
1000111111
====
0001000000
1001001010
====
0001000001
1001001011
====
0001000100
1001001110
====
0001000101
1001001111
====
0001001000
0010000100
====
0001001001
0010000101
====
0001001010
0010000110
====
0001001011
0010000111
====
0001010000
1001011010
====
0001010001
1001011011
====
0001010100
1001011110
====
0001010101
1001011111
====
0001011000
0010010100
====
0001011001
0010010101
====
0001011010
0010010110
====
0001011011
0010010111
====
0001100000
1001101010
====
0001100001
1001101011
====
0001100100
1001101110
====
0001100101
1001101111
====
0001101000
0010100100

In [8]:
v = np.array([[1, 1, 1], 
              [0, 1, 0]])
w = np.array([[0, 0, 0], 
              [1, 0, 0]])
num = 3
print("x + y -2z =c")
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

x + y -2z =c
000
111
====
001
None
====
010
100
====
011
101
====
100
010
====
101
011
====
110
None
====
111
000
====


In [9]:
v = np.array([[0, 0, 0, 1], 
              [1, 0, 1, 0]])
w = np.array([[0, 1, 0, 0], 
              [0, 1, 0, 1]])
num = 4
print("x - z =c")
print("x + y + z + a =c")
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

x - z =c
x + y + z + a =c
0000
None
====
0001
0100
====
0010
None
====
0011
0110
====
0100
0001
====
0101
1010
====
0110
0011
====
0111
None
====
1000
None
====
1001
1100
====
1010
0101
====
1011
1110
====
1100
1001
====
1101
None
====
1110
1011
====
1111
None
====


In [10]:
v = np.array([[1, 0, 1], 
              [0, 1, 0]])
w = np.array([[0, 0, 0], 
              [1, 0, 0]])
num = 3
print("x1+x2-x3=c")
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

x1+x2-x3=c
000
101
====
001
None
====
010
100
====
011
101
====
100
010
====
101
000
====
110
None
====
111
010
====


In [11]:
v = np.array([[0, 1, 1, 0, 0], 
              [0, 1, 0, 1, 0], 
              [0, 0, 0, 0, 0]])
w = np.array([[0, 0, 0, 0, 0], 
              [1, 0, 0, 0, 0], 
              [0, 0, 0, 0, 0]])
num = 5
print("x1+x2-x3=c")
print("x1+x4-x5=c")

for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

x1+x2-x3=c
x1+x4-x5=c
00000
00000
====
00001
00001
====
00010
00010
====
00011
00011
====
00100
00100
====
00101
00101
====
00110
00110
====
00111
00111
====
01000
01000
====
01001
01001
====
01010
01010
====
01011
01011
====
01100
00000
====
01101
00001
====
01110
00010
====
01111
00011
====
10000
01010
====
10001
01011
====
10010
10010
====
10011
10011
====
10100
01110
====
10101
01111
====
10110
10110
====
10111
10111
====
11000
11000
====
11001
11001
====
11010
11010
====
11011
11011
====
11100
10000
====
11101
10001
====
11110
10010
====
11111
10011
====


In [12]:
v = np.array([[0, 1, 1, 0, 0], 
              [0, 1, 0, 1, 0], 
              [0, 0, 0, 0, 1]])
w = np.array([[0, 0, 0, 0, 0], 
              [1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0]])
# v = np.array([[0, 1, 0], 
#               [0, 0, 1]])
# w = np.array([[1, 0, 0], 
#               [1, 0, 0]])
# v = np.array([[0, 1, 1]])
# w = np.array([[1, 0, 0]]) 
num = 5
print("x1+x2-x3=c")
print("x1+x4-x5=c")
# print("x1+x2=c")
# print("x2+x3=c")
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

x1+x2-x3=c
x1+x4-x5=c
00000
01100
====
00001
01000
====
00010
01110
====
00011
01010
====
00100
None
====
00101
01100
====
00110
None
====
00111
01110
====
01000
00001
====
01001
None
====
01010
00011
====
01011
10001
====
01100
00000
====
01101
00001
====
01110
00010
====
01111
00011
====
10000
01010
====
10001
01011
====
10010
11110
====
10011
11010
====
10100
01110
====
10101
01111
====
10110
None
====
10111
11110
====
11000
10001
====
11001
None
====
11010
10011
====
11011
None
====
11100
10000
====
11101
10001
====
11110
10010
====
11111
10011
====


In [13]:
# v = np.array([[0, 1, 1, 0, 0], 
#               [0, 1, 0, 1, 0], 
#               [0, 0, 0, 0, 1]])
# w = np.array([[0, 0, 0, 0, 0], 
#               [1, 0, 0, 0, 0], 
#               [0, 1, 0, 0, 0]])
# v = np.array([[0, 1, 0], 
#               [0, 0, 1]])
# w = np.array([[1, 0, 0], 
#               [1, 0, 0]])
v = np.array([[0, 1, 0, 1], 
              [0, 1, 1, 0]])
w = np.array([[1, 0, 0, 0], 
              [1, 0, 0, 0]]) 
num = 4
print("x1+x2=c")
print("x2+x3+x4=c")
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

x1+x2=c
x2+x3+x4=c
0000
None
====
0001
None
====
0010
None
====
0011
None
====
0100
None
====
0101
1000
====
0110
1000
====
0111
1001
====
1000
0101
====
1001
0111
====
1010
0111
====
1011
None
====
1100
None
====
1101
None
====
1110
None
====
1111
None
====


In [14]:
v = np.array([[0, 1, 1]])
w = np.array([[1, 0, 0]]) 
num = 3
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

000
None
====
001
None
====
010
None
====
011
100
====
100
011
====
101
None
====
110
None
====
111
None
====


In [15]:
import numpy as np
from functools import reduce

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

# 定义计算哈密顿量的函数
def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term3 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) for j in range(n)])
        term4 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

# 例子
v = np.array([[1, 0, 0], 
              [0, 1, 0]])
w = np.array([[0, 1, 0], 
              [0, 0, 1]])
hamiltonian = calculate_hamiltonian(v, w)
print("Hamiltonian:")
print(hamiltonian.real.astype(int))


Hamiltonian:
[[0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 1 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 1 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]]


In [16]:
import numpy as np

def matrix_power(matrix, n):
    """
    计算矩阵的 n 次方
    
    参数：
    matrix: numpy.ndarray，要计算的矩阵
    n: int，次方数
    
    返回值：
    numpy.ndarray，矩阵的 n 次方
    """
    # 检查矩阵是否为方阵
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("输入矩阵必须是方阵")

    # 计算矩阵的 n 次方
    result = np.linalg.matrix_power(matrix, n)
    return result

# 测试
matrix = np.array([[1, 2],
                   [3, 4]])

n = 3
result = matrix_power(matrix, 2)
print(f"{matrix} 的 {n} 次方为:")
print(result)


[[1 2]
 [3 4]] 的 3 次方为:
[[ 7 10]
 [15 22]]
